In [ ]:
import math
import random
from typing import List, Dict, Tuple
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import optuna
import re

SEED = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_PATH = "dataset_vrp/2014-2023_lags_EWM_targets_train.xlsx"
TARGET_LOG_COL = "log_target_next_year"
TEST_YEAR = 2023
TRAIN_MAX_YEAR = 2022
N_TRIALS = 100

def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed()

def mape(y_true, y_pred):
    mask = y_true != 0
    if not np.any(mask): 
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask]))

def smape(y_true, y_pred):
    denom = np.abs(y_true) + np.abs(y_pred)
    mask = denom != 0
    if not np.any(mask): 
        return np.nan
    return 100 * np.mean(2 * np.abs(y_pred[mask] - y_true[mask]) / denom[mask])

def compute_all_metrics(y_log_true, y_log_pred):
    mae_log = mean_absolute_error(y_log_true, y_log_pred)
    mse_log = mean_squared_error(y_log_true, y_log_pred)
    r2_log = r2_score(y_log_true, y_log_pred)
    y_real_true = np.exp(y_log_true)
    y_real_pred = np.exp(y_log_pred)
    mae_real = mean_absolute_error(y_real_true, y_real_pred)
    mse_real = mean_squared_error(y_real_true, y_real_pred)
    rmse_real = math.sqrt(mse_real)
    mape_real = mape(y_real_true, y_real_pred)
    smape_real = smape(y_real_true, y_real_pred)
    return {
        "MAE_log": mae_log,
        "MSE_log": mse_log,
        "R2_log": r2_log,
        "MAE_real": mae_real,
        "MSE_real": mse_real,
        "RMSE_real": rmse_real,
        "MAPE_real": mape_real,
        "SMAPE_real": smape_real
    }


def infer_lag_groups(columns: List[str]) -> Tuple[Dict[str, List[str]], List[str]]:
    lag_pattern = re.compile(r"^(.*)_lag_(\d+)$")
    groups, others = {}, []
    for c in columns:
        m = lag_pattern.match(c)
        if m:
            base, idx = m.group(1), int(m.group(2))
            groups.setdefault(base, []).append((idx, c))
        else:
            others.append(c)
    groups_ordered = {base: [col for _, col in sorted(lst)] for base, lst in groups.items()}

    return groups_ordered, others


class VRPDataset(Dataset):
    def __init__(self, df: pd.DataFrame, seq_groups: Dict[str, List[str]], static_cols: List[str],
                 seq_scalers: Dict[str, StandardScaler] = None, static_scaler: StandardScaler = None, is_train=False):
        self.df = df.reset_index(drop=True)
        self.seq_groups = seq_groups
        self.static_cols = static_cols
        self.seq_names = list(seq_groups.keys())
        self.seq_len = len(seq_groups[self.seq_names[0]]) if self.seq_names else 0

        seq_arrays = [df[seq_groups[base]].values.astype(np.float32) for base in self.seq_names]
        self.seq = np.stack(seq_arrays, axis=1) if seq_arrays else np.zeros((len(df),0,0), np.float32)
        self.static = df[static_cols].values.astype(np.float32) if static_cols else np.zeros((len(df),0), np.float32)
        self.y_log = df[TARGET_LOG_COL].values.astype(np.float32)

        self.seq_scalers = seq_scalers or {}
        if is_train:
            for i, base in enumerate(self.seq_names):
                sc = StandardScaler()
                sc.fit(self.seq[:, i, :])
                self.seq_scalers[base] = sc
        if static_scaler is None and is_train and self.static.shape[1]>0:
            static_scaler = StandardScaler().fit(self.static)
        self.static_scaler = static_scaler

        for i, base in enumerate(self.seq_names):
            sc = self.seq_scalers.get(base)
            if sc: 
                self.seq[:, i, :] = sc.transform(self.seq[:, i, :])
        if self.static_scaler and self.static.shape[1] > 0:
            self.static = self.static_scaler.transform(self.static)

    def __len__(self): 
        return len(self.df)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.seq[idx]).float(), torch.from_numpy(self.static[idx]).float(), torch.tensor(self.y_log[idx]).float()


class ResBlock1D(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, dropout, stride=1):
        super().__init__()
        padding = (kernel_size - 1) // 2

        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size,
                               stride=stride, padding=padding, bias=False)
        self.bn1 = nn.BatchNorm1d(out_ch)

        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size,
                               padding=padding, bias=False)
        self.bn2 = nn.BatchNorm1d(out_ch)

        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()

        self.downsample = None
        if stride != 1 or in_ch != out_ch:
            self.downsample = nn.Sequential(
                nn.Conv1d(in_ch, out_ch, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(out_ch)
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(identity)

        out += identity
        return self.relu(out)


class ResNet1D_Model(nn.Module):
    def __init__(
        self,
        seq_channels,
        seq_len,
        n_filters=64,
        kernel_size=3,
        n_blocks=2,
        dropout=0.1,
        static_size=0,
        mlp_hidden=128
    ):
        super().__init__()

        self.seq_channels = seq_channels

        if seq_channels > 0:
            self.stem = nn.Sequential(
                nn.Conv1d(seq_channels, n_filters, kernel_size=3, padding=1, bias=False),
                nn.BatchNorm1d(n_filters),
                nn.ReLU(inplace=True)
            )
        else:
            self.stem = None

        self.stage1 = self._make_stage(
            n_filters, n_filters, n_blocks, kernel_size, dropout, stride=1
        )
        self.stage2 = self._make_stage(
            n_filters, n_filters * 2, n_blocks, kernel_size, dropout, stride=2
        )
        self.stage3 = self._make_stage(
            n_filters * 2, n_filters * 4, n_blocks, kernel_size, dropout, stride=2
        )

        self.global_pool = nn.AdaptiveAvgPool1d(1)

        mlp_in = (n_filters * 4 if seq_channels > 0 else 0) + static_size

        self.mlp = nn.Sequential(
            nn.Linear(mlp_in, mlp_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_hidden, mlp_hidden // 2),
            nn.ReLU(),
            nn.Linear(mlp_hidden // 2, 1)
        )

    def _make_stage(self, in_ch, out_ch, n_blocks, kernel_size, dropout, stride):
        layers = []
        layers.append(
            ResBlock1D(in_ch, out_ch, kernel_size, dropout, stride=stride)
        )
        for _ in range(1, n_blocks):
            layers.append(
                ResBlock1D(out_ch, out_ch, kernel_size, dropout, stride=1)
            )
        return nn.Sequential(*layers)

    def forward(self, seq, static):
        if self.seq_channels > 0:
            x = self.stem(seq)
            x = self.stage1(x)
            x = self.stage2(x)
            x = self.stage3(x)
            x = self.global_pool(x).squeeze(-1)
        else:
            x = torch.zeros((seq.shape[0], 0), device=seq.device)

        if static.numel() > 0:
            x = torch.cat([x, static], dim=1)

        return self.mlp(x).squeeze(-1)


def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total = 0
    for seq, static, y in loader:
        seq, static, y = seq.to(DEVICE), static.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        loss = criterion(model(seq, static), y)
        loss.backward()
        optimizer.step()
        total += loss.item() * seq.size(0)
    return total / len(loader.dataset)

def eval_model(model, loader, criterion):
    model.eval()
    total, ys, preds = 0, [], []
    with torch.no_grad():
        for seq, static, y in loader:
            seq, static, y = seq.to(DEVICE), static.to(DEVICE), y.to(DEVICE)
            pred = model(seq, static)
            total += criterion(pred, y).item()*seq.size(0)
            ys.append(y.cpu().numpy())
            preds.append(pred.cpu().numpy())
    ys = np.concatenate(ys)
    preds = np.concatenate(preds)
    return total/len(loader.dataset), ys, preds


def objective(trial, df):
    n_filters = trial.suggest_categorical("n_filters", [16, 32, 64, 128])
    kernel_size = trial.suggest_categorical("kernel_size", [3, 5, 7])
    n_blocks = trial.suggest_int("n_blocks", 2, 5)
    dropout = trial.suggest_float("dropout", 0, 0.5)
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-8, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    mlp_hidden = trial.suggest_categorical("mlp_hidden", [32, 64, 128, 256])
    epochs = trial.suggest_int("epochs", 5, 100)

    df_train = df[df.year<=TRAIN_MAX_YEAR].reset_index(drop=True)
    df_test = df[df.year==TEST_YEAR].reset_index(drop=True)
    if len(df_train) < 10: 
        raise optuna.exceptions.TrialPruned()

    seq_groups, others = infer_lag_groups(df.columns.tolist())
    id_like = {"district", "region", "year", "region_encoded", "okrug_encoded"}
    target_cols = {TARGET_LOG_COL, "target_next_year", "delta_target", "delta_target_percent"}
    static_cols = [c for c in others if c not in id_like and c not in target_cols and "_lag_" not in c]

    seq_scalers = {base: StandardScaler().fit(df_train[cols].values.astype(np.float32)) for base,cols in seq_groups.items()}
    static_scaler = StandardScaler().fit(df_train[static_cols].values.astype(np.float32)) if static_cols else None

    train_ds = VRPDataset(df_train,seq_groups,static_cols,seq_scalers,static_scaler)
    test_ds = VRPDataset(df_test,seq_groups,static_cols,seq_scalers,static_scaler)

    train_loader = DataLoader(train_ds,batch_size=batch_size,shuffle=True)
    test_loader = DataLoader(test_ds,batch_size=batch_size,shuffle=False)

    model = ResNet1D_Model(len(seq_groups), train_ds.seq_len, n_filters, kernel_size, n_blocks, dropout, train_ds.static.shape[1], mlp_hidden).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()

    best_state = None
    for epoch in range(1, epochs + 1):
        train_one_epoch(model, train_loader, optimizer, criterion)
        train_loss, _, _ = eval_model(model, train_loader, criterion)
        trial.report(train_loss, epoch)
        if trial.should_prune(): 
            raise optuna.exceptions.TrialPruned()
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}

    model.load_state_dict(best_state)
    _, y_test, y_pred_log = eval_model(model, test_loader, criterion)
    metrics = compute_all_metrics(y_test, y_pred_log)
    trial.set_user_attr("metrics", metrics)
    return metrics["MAE_log"]


def run_hpo(n_trials=N_TRIALS, data_path=DATA_PATH):
    df = pd.read_excel(data_path)
    study = optuna.create_study(direction="minimize")
    study.optimize(lambda t: objective(t, df), n_trials=n_trials, show_progress_bar=True)

    trial = study.best_trial
    print(f"Лучшие параметры: {trial.params}")
    print(f"\nМетрики 2023:")
    for name, value in trial.user_attrs.get("metrics", {}).items():
        print(f"{name}: {value}")
    return study

if __name__=="__main__":
    run_hpo()


[I 2025-12-16 12:14:18,328] A new study created in memory with name: no-name-2bc3ab94-cceb-4b4a-925f-daf465e924ce


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-12-16 12:14:51,515] Trial 0 finished with value: 0.384048193693161 and parameters: {'n_filters': 128, 'kernel_size': 3, 'n_blocks': 5, 'dropout': 0.11315371344546471, 'lr': 2.143421753597418e-05, 'weight_decay': 7.657322158011464e-05, 'batch_size': 64, 'mlp_hidden': 64, 'epochs': 68}. Best is trial 0 with value: 0.384048193693161.
[I 2025-12-16 12:15:22,277] Trial 1 finished with value: 0.40717798471450806 and parameters: {'n_filters': 32, 'kernel_size': 3, 'n_blocks': 5, 'dropout': 0.1502584708052767, 'lr': 0.0006843480966779932, 'weight_decay': 6.10389237511571e-07, 'batch_size': 64, 'mlp_hidden': 256, 'epochs': 77}. Best is trial 0 with value: 0.384048193693161.
[I 2025-12-16 12:15:27,442] Trial 2 finished with value: 0.732684850692749 and parameters: {'n_filters': 128, 'kernel_size': 7, 'n_blocks': 5, 'dropout': 0.4429983770186373, 'lr': 0.00010635935925478696, 'weight_decay': 5.644978252707073e-08, 'batch_size': 32, 'mlp_hidden': 256, 'epochs': 5}. Best is trial 0 with val